## Bert 😊😢
https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english?text=I+am+so+excited



In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
model.to("cuda")

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [7]:

# Continuous input loop
while True:
    user_input = input("Enter your text (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    # Tokenize the input and move tensors to GPU
    inputs = tokenizer(user_input, return_tensors="pt").to("cuda")

    # Perform inference without gradient computation
    with torch.no_grad():
        logits = model(**inputs).logits

    # Get the predicted class ID and corresponding label
    predicted_class_id = logits.argmax().item()
    predicted_label = model.config.id2label[predicted_class_id]

    # Output the predicted label
    print(f"Predicted label: {predicted_label}")

Enter your text (or type 'exit' to quit): I have an exam tomorrow
Predicted label: NEGATIVE
Enter your text (or type 'exit' to quit): I am so excited
Predicted label: POSITIVE
Enter your text (or type 'exit' to quit): exi
Predicted label: NEGATIVE
Enter your text (or type 'exit' to quit): exit


## Translation
https://huggingface.co/marefa-nlp/marefa-mt-en-ar

In [8]:
from transformers import MarianTokenizer, MarianMTModel


In [ ]:
mname = "marefa-nlp/marefa-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)
# Move To GPU
model.to("cuda")


In [11]:
# Continuous input loop
while True:
    user_input = input("Enter your text (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    # English Sample Text
    translated_tokens = model.generate(**tokenizer.prepare_seq2seq_batch([user_input], return_tensors="pt").to('cuda'))

    # Perform inference without gradient computation
    with torch.no_grad():
      translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]


      # Output the predicted label
    print(f"Arabic Trransalation: {translated_text}")


Enter your text (or type 'exit' to quit): hello


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4252: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Arabic Trransalation: ['آلوڤاي']
Enter your text (or type 'exit' to quit): how are you
Arabic Trransalation: ['كيف تكون']
Enter your text (or type 'exit' to quit): i am so happy
Arabic Trransalation: ['أنا سعيد جداً']
Enter your text (or type 'exit' to quit): how was your first day at school?
Arabic Trransalation: ['كيف كان يومك الأول في المدرسة؟']
Enter your text (or type 'exit' to quit): exit


## Gemini

In [13]:
from google.colab import userdata
api_key = userdata.get('GEMINI_KEY')

In [14]:
import google.generativeai as genai

genai.configure(api_key=api_key)

In [15]:
model = genai.GenerativeModel("gemini-1.5-flash")

In [30]:
def summarize_text(prompt):
  response = model.generate_content(prompt)

  summary = response.text

  return summary


In [31]:
num_words = 100


while True:
    # Continuously take user input
    text = input("Enter the text you want summarized (or type 'exit' to quit): ")

    if text.lower() == 'exit':
        print("Exiting the summarization loop.")
        break

    # Create the prompt with the user-provided text and word limit
    prompt = f"""
    Your task is text summarization.
    I'll give you full text. And your job is to summarize it in less than {num_words} words.
    The text is shown below, delimited by triple backticks.
    ```{text}```
    """

    # Call the summarization function (replace with actual implementation)
    summary = summarize_text(prompt)

    # Output the summary
    print(f"Summary (<= {num_words} words): {summary}")

Enter the text you want summarized (or type 'exit' to quit): In recent years, climate change has become one of the most pressing global challenges. The rise in average global temperatures has led to an increase in the frequency and severity of extreme weather events such as hurricanes, wildfires, and droughts. As the polar ice caps melt, sea levels are rising, threatening coastal communities and ecosystems. In addition to environmental impacts, climate change also poses significant risks to human health, agriculture, and infrastructure.  Governments, businesses, and individuals are increasingly recognizing the importance of addressing climate change through both mitigation and adaptation strategies. Mitigation efforts focus on reducing greenhouse gas emissions by transitioning to renewable energy sources, improving energy efficiency, and promoting sustainable land use practices. On the other hand, adaptation strategies aim to enhance resilience to the effects of climate change, such as